# Generate training data

256×169 samples</p>
### Change demand
timestep = 1h, 1 week ->169 timesteps in total</p>
at every timestep, change the nodal demand</p>
### Change topology
Randomly cut the connectivity between two pipes, but preserving the connectivity between each node in the network and source. ->256 topology

### Try single timestep

In [1]:
import wntr
import networkx as nx
import scipy.sparse as sp
import numpy as np 
import random
import tqdm
import os 
import matplotlib.pyplot as plt
import itertools
from collections import Counter 
import networkx as nx
import copy
import pandas as pd
import tensorflow_gnn as tfgnn
import tensorflow as tf

In [2]:
def get_removable_links(inp_file):
    wn = wntr.network.WaterNetworkModel(inp_file)
    removable_links = []
    G = wn.to_graph(wn)
    # Store not-deadend pipes to be removed
    for _, link in wn.links():
        if (link.link_type == 'Pipe' and
            link.start_node.node_type == 'Junction' and
            link.end_node.node_type == 'Junction' and
            G.degree[link.start_node.name] > 1 and
            G.degree[link.end_node.name] > 1):
            removable_links.append(link)
    # To find pairs of links that can be removed together while keeping the network graph connected
    removable_pairs = []
    for (link1, link2) in itertools.combinations(removable_links, 2):
        wnr = copy.deepcopy(wn)  #create a new reference to the same object
        wnr.remove_link(link1)
        wnr.remove_link(link2)
        Gr = wnr.to_graph().to_undirected()
        if nx.is_connected(Gr):
            removable_pairs.append((link1, link2))

    return removable_links, removable_pairs


In [3]:
network = 'ASnet2'
inp_file = network + '.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

# get removable links
removable_links, removable_pairs = get_removable_links(inp_file)
num_nodes = wn.num_nodes
num_links = wn.num_links
num_time = int(wn.options.time.duration / wn.options.time.report_timestep + 1)

In [4]:
n_sims = 256
#𝑉_𝑖=[ 𝐼_𝑖^𝑑, 𝑞_𝑖, 𝐼_𝑖^𝑚,𝐻_𝑖^∗ ] 
#𝐸_𝑝=[𝑖,𝑗,𝑐_𝑝]
#Create data matrix
A = np.zeros((n_sims , num_links, 3))
B = np.zeros((n_sims , num_nodes, 5))
#U = np.zeros((n_sims , num_nodes, 1))


#A = np.zeros((n_sims * num_time, num_links, 3))
#B = np.zeros((n_sims * num_time, num_nodes, 4))
#U = np.zeros((n_sims * num_time, num_nodes, 1))

# Store the randomly chosen pairs of removable links
links2remove = random.sample(removable_pairs, n_sims)

monitor = ['5', '11', '32', '37', '44']

In [5]:
for sim in range(n_sims):
    wn = wntr.network.WaterNetworkModel(inp_file)  # reset value
    wn.options.hydraulic.demand_model = 'DD' #dynamic demand model

    i = 0
    for _, node in wn.nodes():
        node.id = i
        i += 1

    if sim != 0:
        (link1, link2) = links2remove[sim - 1]
        wn.remove_link(link1)
        wn.remove_link(link2)

    i = 0
    for _, link in wn.links():
        A[sim, i, 0] = link.start_node.id
        A[sim, i, 1] = link.end_node.id
        A[sim, i, 2] = 1 / (10.667 *link.length / link.roughness ** 1.852 /link.diameter ** 4.871)
        i += 1
        #A[sim * num_time: (sim + 1) * num_time, i, 0] = link.start_node.id
        #A[sim * num_time: (sim + 1) * num_time, i, 1] = link.end_node.id
        #A[sim * num_time: (sim + 1) * num_time, i, 2] = 1 / (10.667 *
                                                           # link.length / link.roughness ** 1.852 /
                                                           # link.diameter ** 4.871)
        #i += 1
    results = wntr.sim.EpanetSimulator(wn).run_sim(version=2.0)
    head = results.node['head']
    demand = results.node['demand']
    demand = np.maximum(demand, 0)
    #### Single timestep
    demand_s = demand.loc[3600]
    head_s = head.loc[3600]
    #####
    Nd = np.array([1 if node.node_type == 'Junction'
                   else 0 for _, node in wn.nodes()])
    Nh = np.array([1 if node.node_type == 'Junction' and node.name not in monitor else 0 for _, node in wn.nodes()])
    B[sim, :, 0] = Nd
    B[sim, :, 1] = demand_s
    B[sim, :, 2] = Nh
    B[sim, :, 3] = (1 - Nh) * head_s
    B[sim, :, 4] = head_s

    data_dir = 'datasets/asnet2/enforce_5_b4/'

    dataset = {'A': A, 'B': B}

### Change 3-d array to DataFrame

In [6]:
# Assuming your 3-dimensional NumPy array is called 'array_3d'
array_3d_shape = B.shape
new_shape = (array_3d_shape[0]* array_3d_shape[1] ,array_3d_shape[2])

# Reshaping the 3-dimensional array into a 2-dimensional array
B_2d = B.reshape(new_shape)

# Converting the 2-dimensional array into a DataFrame
node_df = pd.DataFrame(B_2d)

array_3d_shape = A.shape
new_shape = (array_3d_shape[0]* array_3d_shape[1] ,array_3d_shape[2])

# Reshaping the 3-dimensional array into a 2-dimensional array
A_2d = A.reshape(new_shape)

# Converting the 2-dimensional array into a DataFrame
edge_df = pd.DataFrame(A_2d)

In [7]:
edge_df.rename(columns={0: 'source', 1: 'target', 2: 'loss_co'},inplace=True)
node_df.rename(columns={0: 'Junction_in', 1: 'demand', 2: 'Measurement_in',3:'measurement',4:'head'},inplace=True)

In [8]:
from sklearn.model_selection import train_test_split

node_train, node_test = train_test_split(node_df,test_size=0.2,random_state=42)
edge_train = edge_df.loc[~((edge_df['source'].isin(node_test.index)) | (edge_df['target'].isin(node_test.index)))]
edge_test = edge_df.loc[(edge_df['source'].isin(node_test.index)) | (edge_df['target'].isin(node_test.index))]

In [11]:
def create_graph_tensor(node_df, edge_df):
    graph_tensor = tfgnn.GraphTensor.from_pieces(
        node_sets={
            "node": tfgnn.NodeSet.from_fields(
                sizes=[len(node_df)],
                features={
                    'Junction_in':np.array(node_df['Junction_in'], dtype='int32').reshape(len(node_df),1),
                    'demand': np.array(node_df['demand'], dtype='float32').reshape(len(node_df),1),
                    'Measurement_in': np.array(node_df['Measurement_in'], dtype='int32').reshape(len(node_df),1),
                    'measurement': np.array(node_df['measurement'], dtype='float32').reshape(len(node_df),1),
                    'head':np.array(node_df['head'], dtype='float32').reshape(len(node_df),1),
                }
            )
        },
        edge_sets={
            "link": tfgnn.EdgeSet.from_fields(
                sizes=[len(edge_df)],
                features={
                    'loss_co': np.array(edge_df['loss_co'], dtype='float32').reshape(len(edge_df),1),
                },
                adjacency=tfgnn.Adjacency.from_indices(
                                          source=("node", np.array(edge_df['source'], dtype='int32')),
                                          target=("node", np.array(edge_df['target'], dtype='int32')),
                                      ))
        }
    )

    return graph_tensor


In [12]:
full_tensor = create_graph_tensor(node_df, edge_df)
train_tensor = create_graph_tensor(node_train, edge_train)
test_tensor = create_graph_tensor(node_test, edge_test)

In [11]:
full_tensor

GraphTensor(
  context=Context(features={}, sizes=[1], shape=(), indices_dtype=tf.int32),
  node_set_names=['node'],
  edge_set_names=['link'])

split off ‘head’ as our target (label).

In [13]:
def node_batch_merge(graph):
    graph = graph.merge_batch_to_components()
    node_features = graph.node_sets['node'].get_features_dict()
    edge_features = graph.edge_sets['link'].get_features_dict()

    label = node_features.pop('head')
    print(label)

    new_graph = graph.replace_features(node_sets={'node': node_features}, edge_sets={'link': edge_features})
    return new_graph, label


def create_dataset(graph, function):
    dataset = tf.data.Dataset.from_tensors(graph)
    dataset = dataset.batch(32)
    return dataset.map(function)

### Train dataset

In [14]:
train_node_dataset = create_dataset(train_tensor, node_batch_merge)
test_node_dataset = create_dataset(test_tensor, node_batch_merge)
full_node_dataset = create_dataset(full_tensor, node_batch_merge)

Tensor("Reshape_5:0", shape=(None, 1), dtype=float32)
Tensor("Reshape_5:0", shape=(None, 1), dtype=float32)
Tensor("Reshape_5:0", shape=(None, 1), dtype=float32)


# Model Building

## Build model inputs

In [15]:
graph_spec = train_node_dataset.element_spec[0]
input_graph = tf.keras.layers.Input(type_spec=graph_spec)

In [16]:
def set_initial_node_state(node_set, node_set_name):
    features = [
        tf.keras.layers.Dense(32, activation="relu")(node_set['Junction_in']),
        tf.keras.layers.Dense(32, activation="relu")(node_set['demand']),
        tf.keras.layers.Dense(32, activation="relu")(node_set['Measurement_in']),
        tf.keras.layers.Dense(32, activation="relu")(node_set['measurement']),
    ]
    return tf.keras.layers.Concatenate()(features)


def set_initial_edge_state(edge_set, edge_set_name):
    features = [
        tf.keras.layers.Dense(32, activation="relu")(edge_set['loss_co']),
    ]
    return tf.keras.layers.Concatenate()(features)


graph = tfgnn.keras.layers.MapFeatures(node_sets_fn=set_initial_node_state,
                                       edge_sets_fn=set_initial_edge_state)(input_graph)


def dense_layer(units=32, l2_reg=0.1, dropout=0.25, activation='relu'):
    regularizer = tf.keras.regularizers.l2(l2_reg)
    return tf.keras.Sequential([
        tf.keras.layers.Dense(units, kernel_regularizer=regularizer, bias_regularizer=regularizer),
        tf.keras.layers.Dropout(dropout)
    ])


### Compile

In [17]:
graph_updates = 2
for i in range(graph_updates):
    graph = tfgnn.keras.layers.GraphUpdate(
        node_sets={
            'node':
            tfgnn.keras.layers.NodeSetUpdate(
                {
                    'link':
                    tfgnn.keras.layers.SimpleConv(message_fn=dense_layer(32),
                                                  reduce_type="sum",
                                                  sender_edge_feature=tfgnn.HIDDEN_STATE,
                                                  receiver_tag=tfgnn.TARGET)
                }, tfgnn.keras.layers.NextStateFromConcat(dense_layer(64)))
        })(graph)


logits = tf.keras.layers.Dense(1)(graph.node_sets["node"][tfgnn.HIDDEN_STATE])

node_model = tf.keras.Model(input_graph, logits)
node_model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
                   loss='mean_squared_error',
                   metrics=['mean_squared_error'])
node_model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [()]                      0         
                                                                 
 map_features (MapFeatures)  ()                        320       
                                                                 
 graph_update (GraphUpdate)  ()                        19552     
                                                                 
 graph_update_1 (GraphUpdate  ()                       11360     
 )                                                               
                                                                 
 input.node_sets (InstancePr  {'node': ()}             0         
 operty)                                                         
                                                                 
 input._get_features_ref_5 (  {'hidden_state': (None,   0    

### Train model

In [18]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10, restore_best_weights=True)

node_model.fit(train_node_dataset.repeat(),
               validation_data=test_node_dataset,
               steps_per_epoch=10,
               epochs=1000,
               callbacks=[es])

Epoch 1/1000
10/10 [==============================] - 2s 76ms/step - loss: 7810353664.0000 - mean_squared_error: 7810353664.0000 - val_loss: 469206784.0000 - val_mean_squared_error: 469206784.0000
Epoch 2/1000
10/10 [==============================] - 1s 51ms/step - loss: 2083572864.0000 - mean_squared_error: 2083572864.0000 - val_loss: 1038971328.0000 - val_mean_squared_error: 1038971328.0000
Epoch 3/1000
10/10 [==============================] - 1s 52ms/step - loss: 838636224.0000 - mean_squared_error: 838636224.0000 - val_loss: 148192592.0000 - val_mean_squared_error: 148192576.0000
Epoch 4/1000
10/10 [==============================] - 1s 52ms/step - loss: 326144352.0000 - mean_squared_error: 326144352.0000 - val_loss: 151231616.0000 - val_mean_squared_error: 151231600.0000
Epoch 5/1000
10/10 [==============================] - 1s 52ms/step - loss: 207326544.0000 - mean_squared_error: 207326512.0000 - val_loss: 29029060.0000 - val_mean_squared_error: 29029038.0000
Epoch 6/1000
10/10 [=

Epoch 45/1000
10/10 [==============================] - 1s 52ms/step - loss: 718980.0000 - mean_squared_error: 718916.6250 - val_loss: 198923.5156 - val_mean_squared_error: 198858.8281
Epoch 46/1000
10/10 [==============================] - 1s 52ms/step - loss: 922770.9375 - mean_squared_error: 922705.1250 - val_loss: 170620.7500 - val_mean_squared_error: 170553.7969
Epoch 47/1000
10/10 [==============================] - 1s 52ms/step - loss: 844824.3125 - mean_squared_error: 844756.3750 - val_loss: 146816.4062 - val_mean_squared_error: 146747.1094
Epoch 48/1000
10/10 [==============================] - 1s 52ms/step - loss: 533327.5000 - mean_squared_error: 533257.1875 - val_loss: 139479.1562 - val_mean_squared_error: 139407.4688
Epoch 49/1000
10/10 [==============================] - 1s 56ms/step - loss: 503016.4062 - mean_squared_error: 502943.6875 - val_loss: 141122.1875 - val_mean_squared_error: 141048.0312
Epoch 50/1000
10/10 [==============================] - 1s 55ms/step - loss: 8673

### Evaluate model

In [19]:
eval_result = node_model.evaluate(test_node_dataset)
print(dict(zip(node_model.metrics_names, eval_result)))

1/1 [==============================] - 0s 68ms/step - loss: 128555.2500 - mean_squared_error: 128464.8047
{'loss': 128555.25, 'mean_squared_error': 128464.8046875}
